In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [10]:
import numpy as np
import pandas as pd
from tensorflow.python.keras.models import Sequential, Model
from tensorflow.python.keras.layers import Dense, LSTM, Dropout, Input
from tensorflow.keras.layers import Bidirectional
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import math
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.preprocessing import MaxAbsScaler,RobustScaler
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint
import datetime
import time
from sklearn.model_selection import train_test_split
import inspect, os

In [19]:
import os, pandas as pd
os.chdir('/content/drive/MyDrive/jiwon')

weather = pd.read_csv('해운대 날씨.csv')
customer = pd.read_csv('해운대 입장객수.csv')

In [5]:
weather.head(5),weather.shape

(           날짜  강수_관측값    기온    습도  체감온도  평균 풍속    평균기압  평균 수온  평균최대파고  평균 파주기
 0  2012-05-01    10.5  20.4  62.1  21.5    6.5  1009.0   15.2     1.2     4.8
 1  2012-05-02     1.0  19.6  70.5  21.4    7.8  1004.0   15.5     2.1     5.7
 2  2012-05-03     0.1  20.7  77.5  22.9    4.1  1004.0   16.4     1.9     6.6
 3  2012-05-04     0.0  19.9  70.7  21.7    4.1  1006.5   16.4     1.6     7.5
 4  2012-05-05     0.0  24.8  47.2  24.7    6.1  1004.9   15.8     1.9     7.8,
 (1618, 10))

In [20]:
# 날짜를 date type으로 변경 후, 나머지는 numeric type으로 변경
weather['날짜'] = pd.to_datetime(weather['날짜'], infer_datetime_format=True)
weather.iloc[:,1:] = weather.iloc[:,1:].apply(pd.to_numeric)

customer['방문일'] = pd.to_datetime(customer['방문일'], infer_datetime_format=True)
customer['방문객수(명)'] = customer['방문객수(명)'].str.replace(",","")
customer.iloc[:,1:] = customer.iloc[:,1:].apply(pd.to_numeric)

In [21]:
# weather data filtering
weather = weather[(weather['날짜'] >= '2021-06-01') & (weather['날짜'] <= '2021-08-31')].reset_index(drop=True)
# customer data drop_duplicates
customer = customer.drop_duplicates().reset_index(drop=True)

In [22]:
# merge data
total_data = pd.merge(weather, customer, left_on='날짜', right_on="방문일", how='inner')
total_data = total_data[['강수_관측값', "기온", "습도", "체감온도", "평균수온", "평균풍속", "평균기압", "평균최대파고", "방문객수(명)"]]

In [11]:
# train/test split
train_x, test_x, train_y, test_y = train_test_split(total_data.iloc[:, :-1], total_data['방문객수(명)'], test_size=0.2)


In [12]:
# minmax scaler
x_mm_scaler = MinMaxScaler()
train_x_scaled = x_mm_scaler.fit_transform(train_x)
test_x_scaled = x_mm_scaler.transform(test_x)


In [13]:
test_x_scaled.shape

(19, 8)

In [14]:
model=Sequential()
model.add(Dense(32,input_dim=8))
model.add(Dense(64))
model.add(Dense(128))
model.add(Dense(64))
model.add(Dense(32))
model.add(Dense(16))
model.add(Dense(1))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                288       
_________________________________________________________________
dense_1 (Dense)              (None, 64)                2112      
_________________________________________________________________
dense_2 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_3 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_4 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_5 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 1

In [15]:
model.compile(loss='mse',optimizer='adam')
model.fit(train_x_scaled,train_y,epochs=50,batch_size=16,validation_split=0.2)

Epoch 1/50
4/4 [==============================] - 2s 110ms/step - loss: 4350161408.0000 - val_loss: 4772526080.0000
Epoch 2/50
4/4 [==============================] - 0s 19ms/step - loss: 4349879808.0000 - val_loss: 4772074496.0000
Epoch 3/50
4/4 [==============================] - 0s 18ms/step - loss: 4349393920.0000 - val_loss: 4771279872.0000
Epoch 4/50
4/4 [==============================] - 0s 19ms/step - loss: 4348597760.0000 - val_loss: 4769859584.0000
Epoch 5/50
4/4 [==============================] - 0s 15ms/step - loss: 4347018240.0000 - val_loss: 4767294976.0000
Epoch 6/50
4/4 [==============================] - 0s 17ms/step - loss: 4343979008.0000 - val_loss: 4762736128.0000
Epoch 7/50
4/4 [==============================] - 0s 19ms/step - loss: 4339492864.0000 - val_loss: 4754779136.0000
Epoch 8/50
4/4 [==============================] - 0s 18ms/step - loss: 4330560000.0000 - val_loss: 4740858368.0000
Epoch 9/50
4/4 [==============================] - 0s 17ms/step - loss: 43165614

In [16]:
loss=model.evaluate(test_x,test_y)
pred_y=model.predict(test_x)

print('loss: ',loss)
print('예상 입장객 수: ', pred_y[-1:])

1/1 [==============================] - 0s 204ms/step - loss: 8114553225216.0000
loss:  8114553225216.0
예상 입장객 수:  [[2944765.]]
